# Region definitions

Uniformly divide the genomes into non-overlapping 10Mb regions

randomly select 500 20Mb region
place causal variants equally spaced (based on the position) in the region 


In [1]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

import numpy as np
import dapgen
import pandas as pd
import dask.array as da
import itertools
import submitit
import json
import matplotlib.pyplot as plt
import admix_genet_cor
import pandas as pd
import admix
from os.path import join
import os

In [2]:
DATA_ROOT_DIR = (
    "/u/project/pasaniuc/pasaniucdata/admixture/projects/PAGE-QC/01-dataset/out/aframr"
)
CHROM = 1

In [3]:
pfile = f"{DATA_ROOT_DIR}/imputed/chr{CHROM}"

geno, df_snp, df_indiv = dapgen.read_pfile(pfile, phase=True, snp_chunk=2048)

In [4]:
np.random.seed(0)
df_region = {"CHROM": [], "START": [], "STOP": []}
n_region = 100
region_size = 20  # in megabase

while len(df_region["START"]) < n_region:
    start = np.random.uniform(
        low=df_snp.POS.min(), high=df_snp.POS.max() - region_size * 1e6
    )
    stop = start + region_size * 1e6
    start, stop = int(start), int(stop)
    n_snp = np.sum((start < df_snp.POS) & (df_snp.POS < stop))
    if n_snp < 1e5:
        continue
    df_region["CHROM"].append(CHROM)
    df_region["START"].append(start)
    df_region["STOP"].append(stop)

df_region = (
    pd.DataFrame(df_region).sort_values(["CHROM", "START"]).reset_index(drop=True)
)
df_region.to_csv("out/regions.txt", sep="\t", index=False)